In [21]:
import pandas as pd
from sqlalchemy import create_engine
import pymongo
import numpy as np
import pprint

## Grabbing the CSV's 

In [2]:
motherShootingsCSV = "motherShootings.csv"
motherDF = pd.read_csv(motherShootingsCSV)
motherDF.head()

,location,date,fatalities,injured,total_victims,gender
0,"San Bernardino, California",2015-12-02,14,21,35,Male & Female
1,"Colorado Springs, Colorado",2015-11-27,3,9,12,M
2,"Colorado Springs, Colorado",2015-10-31,3,0,3,M
3,"Roseburg, Oregon",2015-10-01,9,9,18,Male
4,"Chattanooga, Tennessee",2015-07-16,5,2,7,Male


In [3]:
csv_file = "crimeData.csv"
crime_df = pd.read_csv(csv_file)
crime_df.head()

,ori,year,department_name,total_pop,homs_sum,rape_sum,rob_sum,agg_ass_sum,violent_crime,months_reported,violent_per_100k,homs_per_100k,rape_per_100k,rob_per_100k,agg_ass_per_100k,source,url
0,AZ00717,2015,"Mesa, Ariz.",471034.0,16.0,241.0,408.0,1307.0,1972.0,NaN,418.653431,3.396782,51.164035,86.617951,277.474662,Crime in the U.S. 2015,https://ucr.fbi.gov/crime-in-the-u.s/2015/crim...
1,AZ00723,2015,Phoenix,1559744.0,112.0,1016.0,3020.0,5113.0,9261.0,NaN,593.751282,7.180666,65.138895,193.621517,327.810205,Crime in the U.S. 2015,https://ucr.fbi.gov/crime-in-the-u.s/2015/crim...
2,AZ01003,2015,"Tucson, Ariz.",529675.0,31.0,422.0,1059.0,1960.0,3472.0,NaN,655.496295,5.852645,79.671497,199.933922,370.038231,Crime in the U.S. 2015,https://ucr.fbi.gov/crime-in-the-u.s/2015/crim...
3,CA00109,2015,"Oakland, Calif.",419481.0,85.0,285.0,3290.0,2391.0,6051.0,NaN,1442.496800,20.263135,67.941099,784.302507,569.990059,Crime in the U.S. 2015,https://ucr.fbi.gov/crime-in-the-u.s/2015/crim...
4,CA01005,2015,"Fresno, Calif.",520837.0,39.0,167.0,1012.0,1653.0,2871.0,NaN,551.228119,7.487947,32.063774,194.302632,317.373766,Crime in the U.S. 2015,https://ucr.fbi.gov/crime-in-the-u.s/2015/crim...


## Cleaning Up The Shootings DataFrame
### A bit brute forced

In [4]:
data = [{'fatalities': motherDF.loc[motherDF['date'].str.contains('2015'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('2015'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('2015'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('2014'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('2014'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('2014'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('2013'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('2013'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('2013'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('2012'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('2012'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('2012'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('2011'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('2011'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('2011'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('2010'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('2010'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('2010'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('2009'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('2009'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('2009'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('2008'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('2008'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('2008'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('2007'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('2007'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('2007'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('2006'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('2006'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('2006'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('2005'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('2005'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('2005'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('2004'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('2004'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('2004'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('2003'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('2003'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('2003'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('2002'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('2002'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('2002'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('2001'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('2001'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('2001'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('2000'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('2000'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('2000'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('1999'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('1999'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('1999'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('1998'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('1998'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('1998'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('1997'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('1997'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('1997'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('1996'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('1996'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('1996'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('1995'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('1995'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('1995'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('1994'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('1994'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('1994'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('1993'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('1993'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('1993'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('1992'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('1992'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('1992'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('1991'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('1991'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('1991'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('1990'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('1990'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('1990'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('1989'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('1989'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('1989'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('1988'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('1988'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('1988'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('1987'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('1987'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('1987'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('1986'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('1986'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('1986'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('1985'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('1985'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('1985'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('1984'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('1984'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('1984'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('1983'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('1983'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('1983'), 'total_victims'].sum()},
        {'fatalities': motherDF.loc[motherDF['date'].str.contains('1982'), 'fatalities'].sum(),\
         'injured': motherDF.loc[motherDF['date'].str.contains('1982'), 'injured'].sum(),\
         'total_victims': motherDF.loc[motherDF['date'].str.contains('1982'), 'total_victims'].sum()},
      ]

In [5]:
shootingData = pd.DataFrame(data, index=np.arange(2015, 1981, -1))
shootingData.head()

,fatalities,injured,total_victims
2015,46,43,89
2014,18,28,46
2013,35,13,48
2012,71,80,151
2011,19,21,40


In [6]:
# Add the years from the index. 
shootingData['years'] = shootingData.index
shootingData.head()

,fatalities,injured,total_victims,years
2015,46,43,89,2015
2014,18,28,46,2014
2013,35,13,48,2013
2012,71,80,151,2012
2011,19,21,40,2011


## Cleaning Up the Crime Data 

In [7]:
new_crime_df = crime_df[['year', 'department_name', 'violent_crime', 'total_pop' ]].copy()
new_crime_df.head()

,year,department_name,violent_crime,total_pop
0,2015,"Mesa, Ariz.",1972.0,471034.0
1,2015,Phoenix,9261.0,1559744.0
2,2015,"Tucson, Ariz.",3472.0,529675.0
3,2015,"Oakland, Calif.",6051.0,419481.0
4,2015,"Fresno, Calif.",2871.0,520837.0


In [8]:
crime_dict = new_crime_df.to_dict('self')
crime_dict

{'year': 0       2015
 1       2015
 2       2015
 3       2015
 4       2015
 5       2015
 6       2015
 7       2015
 8       2015
 9       2015
 10      2015
 11      2015
 12      2015
 13      2015
 14      2015
 15      2015
 16      2015
 17      2015
 18      2015
 19      2015
 20      2015
 21      2015
 22      2015
 23      2015
 24      2015
 25      2015
 26      2015
 27      2015
 28      2015
 29      2015
         ... 
 2316    1982
 2317    1982
 2318    1982
 2319    1982
 2320    1982
 2321    1982
 2322    1982
 2323    1982
 2324    1982
 2325    1982
 2326    1982
 2327    1982
 2328    1982
 2329    1982
 2330    1982
 2331    1982
 2332    1982
 2333    1982
 2334    1982
 2335    1982
 2336    1982
 2337    1982
 2338    1982
 2339    1982
 2340    1982
 2341    1982
 2342    1982
 2343    1982
 2344    1982
 2345    1982
 Name: year, Length: 2346, dtype: int64,
 'department_name': 0                      Mesa, Ariz.
 1                          Phoenix
 2    

In [9]:
crime_year = new_crime_df.groupby('year')
crime_year.sum()
# crime_year.sort_values(by='index', ascending = False)

,violent_crime,total_pop
year,,
1982,1930713.0,48338985.0
1983,1850503.0,48900834.0
1984,1886809.0,49055031.0
1985,1969260.0,49191535.0
1986,2205660.0,49677351.0
1987,2189833.0,50251736.0
1988,2292660.0,50770146.0
1989,2433172.0,50779917.0
1990,2686096.0,50567789.0


In [10]:
crimeDF = crime_year.sum()
crimeDFyear = crimeDF.sort_values(by='year', ascending=False)
crimeDFyear

,violent_crime,total_pop
year,,
2015,1623131.0,62560261.0
2014,1574333.0,61828407.0
2013,1590805.0,61092218.0
2012,1646202.0,60479280.0
2011,1630009.0,59819081.0
2010,1689509.0,59223241.0
2009,1789882.0,60385217.0
2008,1892222.0,59558390.0
2007,1934105.0,58554165.0


## Combining the Two DataFrames

In [11]:
shootingData['Violent Crime'] = crimeDFyear['violent_crime']

In [12]:
shootingData

,fatalities,injured,total_victims,years,Violent Crime
2015,46,43,89,2015,1623131.0
2014,18,28,46,2014,1574333.0
2013,35,13,48,2013,1590805.0
2012,71,80,151,2012,1646202.0
2011,19,21,40,2011,1630009.0
2010,9,2,11,2010,1689509.0
2009,39,38,77,2009,1789882.0
2008,17,24,41,2008,1892222.0
2007,53,32,85,2007,1934105.0
2006,21,7,28,2006,1957346.0


In [13]:
shootingData['Population in Metro Areas'] = crimeDFyear['total_pop']

In [14]:
shootingData

,fatalities,injured,total_victims,years,Violent Crime,Population in Metro Areas
2015,46,43,89,2015,1623131.0,62560261.0
2014,18,28,46,2014,1574333.0,61828407.0
2013,35,13,48,2013,1590805.0,61092218.0
2012,71,80,151,2012,1646202.0,60479280.0
2011,19,21,40,2011,1630009.0,59819081.0
2010,9,2,11,2010,1689509.0,59223241.0
2009,39,38,77,2009,1789882.0,60385217.0
2008,17,24,41,2008,1892222.0,59558390.0
2007,53,32,85,2007,1934105.0,58554165.0
2006,21,7,28,2006,1957346.0,58744063.0


In [15]:
combinedDF = shootingData.rename(columns={'fatalities': 'Fatalities',
                             'injured': 'Injured',
                             'total_victims': 'Total Victims',
                             'years': 'Years'})

In [16]:
combinedDF

,Fatalities,Injured,Total Victims,Years,Violent Crime,Population in Metro Areas
2015,46,43,89,2015,1623131.0,62560261.0
2014,18,28,46,2014,1574333.0,61828407.0
2013,35,13,48,2013,1590805.0,61092218.0
2012,71,80,151,2012,1646202.0,60479280.0
2011,19,21,40,2011,1630009.0,59819081.0
2010,9,2,11,2010,1689509.0,59223241.0
2009,39,38,77,2009,1789882.0,60385217.0
2008,17,24,41,2008,1892222.0,59558390.0
2007,53,32,85,2007,1934105.0,58554165.0
2006,21,7,28,2006,1957346.0,58744063.0


In [17]:
combinedDF = combinedDF[['Years', 'Fatalities', 'Injured', 'Total Victims', 'Violent Crime', 'Population in Metro Areas']]
combinedDF

,Years,Fatalities,Injured,Total Victims,Violent Crime,Population in Metro Areas
2015,2015,46,43,89,1623131.0,62560261.0
2014,2014,18,28,46,1574333.0,61828407.0
2013,2013,35,13,48,1590805.0,61092218.0
2012,2012,71,80,151,1646202.0,60479280.0
2011,2011,19,21,40,1630009.0,59819081.0
2010,2010,9,2,11,1689509.0,59223241.0
2009,2009,39,38,77,1789882.0,60385217.0
2008,2008,17,24,41,1892222.0,59558390.0
2007,2007,53,32,85,1934105.0,58554165.0
2006,2006,21,7,28,1957346.0,58744063.0


## Export into Mongo

In [29]:
# create connection
conn = 'mongodb://localhost:27017'
# connection to instance
client = pymongo.MongoClient(conn)
# drop database 
# create database
db = client.combinedCrimeShooting
# Drop Collection shootings
db.combinedData.drop()
# Create shootings collection then insert stuff
db.combinedData.insert_many(combinedDF.to_dict('records'))
print('Project Done')

Project Done


In [38]:
pprint.pprint(db.combinedData.find_one())

{'Fatalities': 46,
 'Injured': 43,
 'Population in Metro Areas': 62560261.0,
 'Total Victims': 89,
 'Violent Crime': 1623131.0,
 'Years': 2015,
 '_id': ObjectId('5d65e9095eb05e9fd514de7b')}
